# Generate Consistent Images for the CNN

*The functions `multiply_image` and `multiply_image_backward` are based on the code Chris Birchall used to [find Wally](https://tech.ovoenergy.com/cheating-at-wheres-wally/).*

*Using [this resource](https://note.nkmk.me/en/python-opencv-pillow-image-size/) to leverage `opencv` functionality to automatically-detect image size.*

*Using [imagej](https://imagej.nih.gov/ij/) to identify the location of each kithara and write out x-y bounding box coordinates for each.*

In [27]:
import cv2
import glob
import os.path
import pandas as pd

### Test out extracting the image size

In [25]:
# get the height, width, and color of an image
# read in the image
test_img = cv2.imread("../data/test/IMAG0009.jpg")
# assign the results to variables y (height), x (width), and c (color)
y, x, c = test_img.shape

# # alternatively, select specific values in the array
# x_max = test_img.shape[1]
# y_max = test_img.shape[0]

# show the type of the variable created and return some stats
print(type(test_img))
print(test_img.shape)
print(type(test_img.shape))
print(y)
print(x)
print(c)

<class 'numpy.ndarray'>
(7680, 10032, 3)
<class 'tuple'>
7680
10032
3


### Test out extracting the kithara size and location 

In [46]:
# read in the csv with the kithara location data
test_kithara = pd.read_csv('../data/coordinates/IMAG0009.csv').iloc[:, 1:]
test_kithara

x_top_left  y_top_left  x_top_right  y_top_right  x_bottom_left  \
0        7616        3704         8128         3704           7616   

   y_bottom_left  x_bottom_right  y_bottom_right  
0           4536            8128            4536

In [54]:
# get the min and max x and y coordinates, plus the x distance and y distance
kx_min = test_kithara.x_top_left.values[0]
kx_max = test_kithara.x_bottom_right.values[0]
ky_min = test_kithara.y_top_left.values[0]
ky_max = test_kithara.y_bottom_right.values[0]
kx_size = kx_max - kx_min
ky_size = ky_max - ky_min
print('width (x): ' + kx_size.astype(str))
print('height (y): ' + ky_size.astype(str))

width (x): 512
height (y): 832


### Write a couple of functions to cut up each image into multiple equally-sized, square chunks

*Since images are not necessarily going to be evenly-dissectable based on the size of the kithara for each, I'm taking two tactics to ensure full-coverage:*
* Chopping each image up starting from the top left (`multiply_image`) and bottom right (`multiply_image_backward`) corners
* Shifting the x and y start coordinates by 1/2 of the kithara image size at a time

*To tag whether or not an image includes a full or partial kithara, I'm doing the following:*
* Taking the x-y coordinates of the kithara image location (generated using `imagej`) and, if the image falls within those coordinates, tagging it as a kithara

*To clean up the data generated, I'm doing the following:*
* Deleting any non-square image from the folder with the cropped images
* Resizing all of the square images to the same dimensions

In [24]:
# for each image in the folder
# get the height and width of each image
# then iterate over the image and create 1000 x 1000 pixel squares
def multiply_image(name):
    img = cv2.imread("../data/test/{}.jpg".format(name))
    y_max, x_max, _ = img.shape   
    for y in range(0, y_max, 500):
        start_y = y
        end_y = y + 1000 
        for x in range(0, x_max, 500):
            start_x = x
            end_x = x + 1000
            cropped_img = img[start_y:end_y, start_x:end_x]
            cv2.imwrite("../data/cropped_images/{}-{}-{}.png".format(name, y, x), cropped_img)

In [26]:
# go over the image but in reverse and create 1000 x 1000 pixel squares
def multiply_image_backward(name):
    # read in the image
    img = cv2.imread("../data/test/{}.jpg".format(name))
    # get the size of the image
    y_max, x_max, _ = img.shape
    # incrementally move backward, starting at the bottom R corner
    for y in range(y_max, 0, -500):
        if y >= 1000:
            start_y = y - 1000
            end_y = y
        else:
            start_y = 0
            end_y = y
        for x in range(x_max, 0, -500):
            if x >= 1000:
                start_x = x - 1000
                end_x = x
            else:
                start_x = 0
                end_x = x
            cropped_img = img[start_y:end_y, start_x:end_x]
            cv2.imwrite("../data/cropped_images/back-{}-{}-{}.png".format(name, end_y, end_x), cropped_img)
    
for path in glob.glob("../data/test/*.jpg"):
    name = os.path.splitext(os.path.basename(path))[0]


In [33]:
# run the functions over the image folder    
for path in glob.glob("../data/test/*.jpg"):
    name = os.path.splitext(os.path.basename(path))[0]
    multiply_image(name)
    multiply_image_backward(name)

In [ ]:
# Need to also:
# get rid of incorrectly-sized images
# create a variable to store the kithara size
# create a variable to store the kithara location and name the file differently if it falls within the kithara range